In [18]:
import datetime
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
import warnings
import time
import pandas as pd
import numpy as np
import os
from sklearn.metrics import roc_auc_score


In [107]:
train_df = pd.read_csv('../temp/fusai_train_df_all.csv')
test_df = pd.read_csv('../temp/fusai_test_df_all.csv')


/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (59,87,109,110,125) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (84,122) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [108]:
chusai_QY_xinxi_train_df = pd.read_csv('../../chusai/data/train/企业基本信息&高管信息&投资信息.csv')
fusai_QY_xinxi_train_df = pd.read_csv('../data/train/企业基本信息&高管信息&投资信息.csv')
QY_xinxi_train_df = pd.concat([chusai_QY_xinxi_train_df, fusai_QY_xinxi_train_df])
QY_xinxi_train_df.rename(columns={'企业名称':'qymc', '注册号':'zch', '统一社会信用代码':'tyshxydm', '注册资金':'zczj', '注册资本(金)币种名称':'zczbbzmc', 
                                  '企业(机构)类型名称':'qyjglxmc', '行业门类代码':'hymldm', '成立日期':'clrq', '核准日期':'hzrq', '住所所在地省份':'zsszdsf', 
                                  '姓名':'xm', '法定代表人标志':'fddbrbz', '首席代表标志':'sxdbbz', '职务':'zw', '投资人':'tzr', '出资比例':'czbl'}, inplace=True)
after_train_df = QY_xinxi_train_df[['qymc', 'zch', 'tyshxydm', 'zczj', 'zczbbzmc', 'qyjglxmc', 'hymldm', 'clrq', 'hzrq', 'zsszdsf']]
after_train_df.drop_duplicates(['qymc'], keep='first', inplace=True)
train_suzhou_set = set(after_train_df[after_train_df.zsszdsf == '江苏省']['qymc'])
train_df['is_suzhou'] = train_df['qymc'].map(lambda x : 1 if x in train_suzhou_set else 0)

QY_xinxi_test_df = pd.read_csv('../data/test/企业基本信息&高管信息&投资信息.csv')
QY_xinxi_test_df.rename(columns={'企业名称':'qymc', '注册号':'zch', '统一社会信用代码':'tyshxydm', '注册资金':'zczj', '注册资本(金)币种名称':'zczbbzmc', 
                                  '企业(机构)类型名称':'qyjglxmc', '行业门类代码':'hymldm', '成立日期':'clrq', '核准日期':'hzrq', '住所所在地省份':'zsszdsf',
                                  '姓名':'xm', '法定代表人标志':'fddbrbz', '首席代表标志':'sxdbbz', '职务':'zw', '投资人':'tzr', '出资比例':'czbl'}, inplace=True)
after_test_df = QY_xinxi_test_df[['qymc', 'zch', 'tyshxydm', 'zczj', 'zczbbzmc', 'qyjglxmc', 'hymldm', 'clrq', 'hzrq', 'zsszdsf']]
after_test_df.drop_duplicates(['qymc'], keep='first', inplace=True)
print(len(after_test_df))
suzhou_set = set(after_test_df[after_test_df.zsszdsf == '江苏省']['qymc'])
print(len(suzhou_set))
test_df['is_suzhou'] = test_df['qymc'].map(lambda x : 1 if x in suzhou_set else 0)
print(len(test_df[test_df.is_suzhou == 1]))
print(len(test_df))

categorical_feature_shixin = ['zsszdsf_encoder', 'qyjglxmc_encoder', 'hymldm_encoder', 
                       'zczbbzmc_encoder', 'fzjglx', 'fzhydm', 'fzhyml_encoder', 'fzjgzt',
                      'fzjgsf', 'qylxmc_encoder', 'xzgh', 'sshydm_encoder', 'zl_encoder', 
                             'fzjgmc_encoder', ]

categorical_feature_chufa = ['zsszdsf_encoder', 'qyjglxmc_encoder', 'hymldm_encoder', 
                       'zczbbzmc_encoder', 'fzjglx', 'fzhydm', 'fzhyml_encoder', 'fzjgzt',
                      'fzjgsf', 'qylxmc_encoder', 'xzgh', 'sshydm_encoder', 'zl_encoder',  'fzjgmc_encoder', 
                            ]


/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


17534
14802
14802
17534


/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [109]:
FRXZXKZX_test_df = pd.read_csv('../data/test/法人行政许可注（撤、吊）销信息.csv')
fr_qymc_set = set(FRXZXKZX_test_df['企业名称'])

test_df['is_fr'] = test_df['qymc'].map(lambda x : 1 if x in fr_qymc_set else 0)

QYFZCHRD_test_df = pd.read_csv('../data/test/企业非正常户认定.csv')
fzc_qymc_set = set(QYFZCHRD_test_df['企业名称'])

test_df['is_fzc'] = test_df['qymc'].map(lambda x : 1 if x in fzc_qymc_set else 0)


In [110]:
FZJGXX_test_df = pd.read_csv('../data/test/分支机构信息.csv')
fz_qymc_set = set(FZJGXX_test_df['企业名称'])

test_df['is_fz'] = test_df['qymc'].map(lambda x : 1 if x in fz_qymc_set else 0)


In [111]:
SDBDJQQJMCF_test_df = pd.read_csv('../data/test/双打办打击侵权假冒处罚案件信息.csv')
qq_qymc_set = set(SDBDJQQJMCF_test_df['企业名称'])

test_df['is_qq'] = test_df['qymc'].map(lambda x : 1 if x in qq_qymc_set else 0)


In [112]:
ZPSJ_test_df = pd.read_csv('../../check/data/test/招聘数据.csv')
zp_qymc_set = set(ZPSJ_test_df['企业名称'])

test_df['is_zp'] = test_df['qymc'].map(lambda x : 1 if x in zp_qymc_set else 0)


In [113]:
lake_rule1_test_df = pd.read_csv('../../check/data/test/企业基本信息&高管信息&投资信息.csv')
temp = lake_rule1_test_df[['企业名称','统一社会信用代码']].groupby(['企业名称'], as_index=False)['统一社会信用代码'].agg({'is_tyshxydm_many':'nunique'})
temp.rename(columns={'企业名称':'qymc'}, inplace=True)
test_df = pd.merge(test_df,temp,on=['qymc'],how='left')

lake_rule2_test_df = pd.read_csv('../../check/data/test/企业基本信息&高管信息&投资信息.csv')
temp = lake_rule2_test_df[['企业名称','注册号']].groupby(['企业名称'], as_index=False)['注册号'].agg({'is_zch_many':'nunique'})
temp.rename(columns={'企业名称':'qymc'}, inplace=True)
test_df = pd.merge(test_df,temp,on=['qymc'],how='left')


In [114]:
train_df['shixin_weight'] = 1
train_df.loc[train_df.is_chufa == 1, 'shixin_weight'] = 2


In [115]:
shixin_fea = ['zczj', 'fzjgsf', 'fzjgzt', 'fzhydm', 'fzjgqx', 'fzjglx', 'sjly', 'rwbh', 
#               'zckbzj', 
       'QYFZCHRD_yynx', 'sshydm_encoder', 'xzgh', 'qyjd', 'qywd', 
#        'QYBZRYXX_number', 'QYFZCHRD_number', 'QYSWDJXX_number', 'FRXZXKZX_number',
#        'SDBDJQQJMCF_number', 'SGSFRXZXKXX_number', 'XKZZNJXX_number',
#        'ZPSJ_number', 'ZZDJBGXX_number', 'QYJL_number', 
      'zczbbzmc_encoder', 'hymldm_encoder', 'zsszdsf_encoder', 'fzhyml_encoder',
       'xxtgbmmc_encoder', 'zl_encoder', 'sszj_encoder', 'fzjgmc_encoder',
       'qyjglxmc_encoder', 'qylxmc_encoder',
              
#        'QYFZCHRD_gljg_encoder', 'QYFZCHRD_djzclx_encoder', 'QYFZCHRD_swglm_encoder',
        
#         'SGSFRXZXKXX_dfbm', 'SGSFRXZXKXX_xknr_encoder', 'SGSFRXZXKXX_xmmc_encoder', 'SGSFRXZXKXX_splb_encoder', 
#        'SGSFRXZXKXX_xkjg_encoder', 'SGSFRXZXKXX_xysyfw_encoder', 'SGSFRXZXKXX_sjzt2_encoder', 
        
#     'XKZZNJXX_njnd_encoder', 'XKZZNJXX_njjgmc_encoder', 'XKZZNJXX_njsxmc_encoder', 
        
#     'ZZDJBGXX_zzmc_encoder', 'ZZDJBGXX_zyfw_encoder', 'ZZDJBGXX_rdjgqc_encoder', 'ZZDJBGXX_zzzsbh_encoder', 
      
#     'QY_tzrgs', 'QY_zwzls', 'QY_fddbrbz_gs', 'QY_sxdbbz_gs', 
              
        'fzsw_deltaDay', 'clhz_deltaDay', 
       'clrq_month', 'clrq_day', 'hzrq_year', 'hzrq_month', 'hzrq_day', 'clrq_year',
              
       'ZPSJ_zprs_max', 'ZPSJ_zprs_min', 'ZPSJ_zprs_mean', 'ZPSJ_zprs_std',
        
    'ZZDJBGXX_jzsx_deltaDay', 'zzsxcl_deltaDay', 'zzjzcl_deltaDay', 'zzsxhz_deltaDay', 'zzjzhz_deltaDay', 
              
    'jgdzzs_encoder', 'jyfw_encoder', 'sshymc_encoder',
    'XKZZNJXX_xxtgbmmc_encoder', 'QYFZCHRD_zcdz_encoder', 
     'QYFZCHRD_rwbh_encoder', 'QYFZCHRD_xxtgbmmc_encoder',
            
              'jgdzzs_top3_encoder', 'jgdzzs_top6_encoder',
#  'QYSWDJXX_fddbrzjmc_encoder', 'QYSWDJXX_shdw_encoder',
#  'QYSWDJXX_qy_encoder', 'QYSWDJXX_shjg_encoder', 'QYSWDJXX_djzclx_encoder',
              
#     'SGSFRXZXKXX_xzxkbm_encoder', 'SGSFRXZXKXX_xkjdswh_encoder', 'SGSFRXZXKXX_xxtgbmmc_encoder',

#       'ZPSJ_zylb_kinds', 'ZPSJ_gzjy_kinds',
#        'ZPSJ_gzdd_kinds', 'ZPSJ_zprs_kinds', 'ZPSJ_zwyx_kinds',
#        'ZPSJ_zdxl_kinds', 'ZPSJ_zprq_kinds',
              
#         'SGSFRXZXKXX_xkjdjz_deltaDay',
#       'SGSFRXZXKXX_xkjzq_year', 'SGSFRXZXKXX_xkjzq_month', 'SGSFRXZXKXX_xkjzq_day', 
#       'SGSFRXZXKXX_xkjdrq_year', 'SGSFRXZXKXX_xkjdrq_month', 'SGSFRXZXKXX_xkjdrq_day',  
    
       
#        'clxkjz_deltaDay', 'hzxkjz_deltaDay', 'clxkjd_deltaDay', 'hzxkjd_deltaDay', 
#         'njhz_deltaDay', 'njcl_deltaDay', 
#        'XKZZNJXX_njrq_year', 'XKZZNJXX_njrq_month', 'XKZZNJXX_njrq_day', 
#        'ZZDJBGXX_zzsxq_year', 'ZZDJBGXX_zzsxq_month', 'ZZDJBGXX_zzsxq_day', 
#        'ZZDJBGXX_zzjzq_year', 'ZZDJBGXX_zzjzq_month', 'ZZDJBGXX_zzjzq_day', 
       
       
#        'czbl_max', 'czbl_min', 'czbl_mean', 'czbl_std',
       
#        'fzrq_encoder', 'bghzrq_encoder',
#      'gszch_encoder', 'fddbrzjhm_encoder', 'zzjgdm_encoder',
       
#        'ZPSJ_wzmc_max', 'ZPSJ_wzmc_min', 'ZPSJ_wzmc_mean',
#  'ZPSJ_wzmc_std', 'ZPSJ_gzjy_max', 'ZPSJ_gzjy_min', 'ZPSJ_gzjy_mean',
#  'ZPSJ_gzjy_std', 'ZPSJ_gzdd_max', 'ZPSJ_gzdd_min', 'ZPSJ_gzdd_mean',
#  'ZPSJ_gzdd_std', 'ZPSJ_zylb_max', 'ZPSJ_zylb_min', 'ZPSJ_zylb_mean',
#  'ZPSJ_zylb_std', 'ZPSJ_zwyx_max', 'ZPSJ_zwyx_min', 'ZPSJ_zwyx_mean',
#  'ZPSJ_zwyx_std', 'ZPSJ_zdxl_max', 'ZPSJ_zdxl_min', 'ZPSJ_zdxl_mean',
#  'ZPSJ_zdxl_std', 'ZPSJ_ywzj_max', 'ZPSJ_ywzj_min', 'ZPSJ_ywzj_mean',
#  'ZPSJ_ywzj_std',
       
#        'SGSFRXZXKXX_is_shixin_sjly_rate', 'SGSFRXZXKXX_is_chufa_sjly_rate',
#  'SGSFRXZXKXX_is_shixin_cjrID_rate', 'SGSFRXZXKXX_is_chufa_cjrID_rate',
#  'SGSFRXZXKXX_is_shixin_xxtgbmbm_rate',
#  'SGSFRXZXKXX_is_chufa_xxtgbmbm_rate', 'SGSFRXZXKXX_is_shixin_splb_rate',
#  'SGSFRXZXKXX_is_chufa_splb_rate', 'SGSFRXZXKXX_is_shixin_dfbm_rate',
#  'SGSFRXZXKXX_is_chufa_dfbm_rate', 'SGSFRXZXKXX_is_shixin_xkjg_rate',
#  'SGSFRXZXKXX_is_chufa_xkjg_rate',
       
      ]



In [116]:
temp_shixin_df = train_df[train_df.is_shixin == 1].copy()
temp_notshixin_df = train_df[(train_df.is_shixin == 0)].copy()
# temp_notshixin_df = temp_notshixin_df.sample(frac=0.9, random_state=2018, replace=True)
temp_train_df_list = list()
skf_dataset = StratifiedKFold(n_splits=4, random_state=0, shuffle=True)
for index, (train_index, test_index) in enumerate(skf_dataset.split(temp_notshixin_df[shixin_fea], temp_notshixin_df['is_shixin'])):
    temp = temp_notshixin_df.iloc[test_index].copy()
    temp = pd.concat([temp, temp_shixin_df])
    temp.reset_index(inplace=True)
    temp_train_df_list.append(temp)


In [117]:
lgb_model_shixin = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=127, reg_alpha=6, reg_lambda=10, max_bin=425,
    max_depth=-1, n_estimators=5000, objective='binary',
#     subsample=0.8, 
    colsample_bytree=1, subsample_freq=1,
    learning_rate=0.02, random_state=2018, n_jobs=-1,min_child_samples=5
)

skf = StratifiedKFold(n_splits=10, random_state=2018, shuffle=True)
for i in range(4):
    best_score = []
    temp_train_df = temp_train_df_list[i]
    test_df['FORTARGET1_round' + str(i)] = 0
    number = 0
    for index, (train_index, test_index) in enumerate(skf.split(temp_train_df[shixin_fea], temp_train_df['is_shixin'])):
        weight = temp_train_df['shixin_weight'].iloc[train_index].tolist()
        lgb_model_shixin.fit(temp_train_df[shixin_fea].iloc[train_index], temp_train_df['is_shixin'][train_index],
                             sample_weight=weight,
                             categorical_feature=categorical_feature_shixin,
                      eval_set=[(temp_train_df[shixin_fea].iloc[train_index], temp_train_df['is_shixin'][train_index]),
                                (temp_train_df[shixin_fea].iloc[test_index], temp_train_df['is_shixin'][test_index])], early_stopping_rounds=200, eval_metric='auc')
        if lgb_model_shixin.best_score_['valid_1']['auc'] > 0.9:
            best_score.append(lgb_model_shixin.best_score_['valid_1']['auc'])
            print(best_score)
            test_pred = lgb_model_shixin.predict_proba(test_df[shixin_fea], num_iteration=lgb_model_shixin.best_iteration_)[:, 1]
            print('test mean:', test_pred.mean())
            test_df['FORTARGET1_round' + str(i)] = test_df['FORTARGET1_round' + str(i)] + test_pred
            number = number + 1
    print(number)
    print(np.mean(best_score))
    test_df['FORTARGET1_round' + str(i)] = test_df['FORTARGET1_round' + str(i)] / number


/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['fzhydm', 'fzhyml_encoder', 'fzjglx', 'fzjgmc_encoder', 'fzjgsf', 'fzjgzt', 'hymldm_encoder', 'qyjglxmc_encoder', 'qylxmc_encoder', 'sshydm_encoder', 'xzgh', 'zczbbzmc_encoder', 'zl_encoder', 'zsszdsf_encoder']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	valid_0's auc: 0.628989	valid_1's auc: 0.606066
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's auc: 0.628989	valid_1's auc: 0.606066
[3]	valid_0's auc: 0.64751	valid_1's auc: 0.599324
[4]	valid_0's auc: 0.782377	valid_1's auc: 0.794589
[5]	valid_0's auc: 0.814339	valid_1's auc: 0.825519
[6]	valid_0's auc: 0.818854	valid_1's auc: 0.831012
[7]	valid_0's auc: 0.81559	valid_1's auc: 0.822081
[8]	valid_0's auc: 0.818558	valid_1's auc: 0.826626
[9]	valid_0's auc: 0.82084	valid_1's auc: 0.828624
[10]	valid_0's auc: 0.821266	valid_1's auc: 0.83008
[11]	valid_0's auc: 0.82084	valid_1's auc: 0.828624
[12]	valid_0's auc: 0.821238	valid_1's auc: 0.830022
[13]	valid_0's auc: 0.821398	valid_1's auc: 0.830621
[14]	valid_0's auc: 0.821398	valid_1's auc: 0.830621
[15]	valid_0's auc: 0.821398	valid_1's auc: 0.830621
[16]	valid_0's auc: 0.821398	valid_1's auc: 0.830621
[17]	valid_0's auc: 0.821398	valid_1's auc: 0.830621
[18]	valid_0's auc: 0.821398	valid_1's auc: 0.8306

[161]	valid_0's auc: 0.934509	valid_1's auc: 0.925513
[162]	valid_0's auc: 0.934861	valid_1's auc: 0.925447
[163]	valid_0's auc: 0.935468	valid_1's auc: 0.925854
[164]	valid_0's auc: 0.936024	valid_1's auc: 0.926337
[165]	valid_0's auc: 0.936313	valid_1's auc: 0.926325
[166]	valid_0's auc: 0.936888	valid_1's auc: 0.926816
[167]	valid_0's auc: 0.937286	valid_1's auc: 0.926961
[168]	valid_0's auc: 0.93746	valid_1's auc: 0.926812
[169]	valid_0's auc: 0.938003	valid_1's auc: 0.927244
[170]	valid_0's auc: 0.938185	valid_1's auc: 0.927194
[171]	valid_0's auc: 0.938706	valid_1's auc: 0.927627
[172]	valid_0's auc: 0.93914	valid_1's auc: 0.927885
[173]	valid_0's auc: 0.939582	valid_1's auc: 0.928293
[174]	valid_0's auc: 0.939883	valid_1's auc: 0.928493
[175]	valid_0's auc: 0.940324	valid_1's auc: 0.928917
[176]	valid_0's auc: 0.940511	valid_1's auc: 0.928859
[177]	valid_0's auc: 0.940688	valid_1's auc: 0.929026
[178]	valid_0's auc: 0.941073	valid_1's auc: 0.929517
[179]	valid_0's auc: 0.94142	v

[315]	valid_0's auc: 0.968181	valid_1's auc: 0.938614
[316]	valid_0's auc: 0.968355	valid_1's auc: 0.938698
[317]	valid_0's auc: 0.968512	valid_1's auc: 0.938689
[318]	valid_0's auc: 0.968665	valid_1's auc: 0.938764
[319]	valid_0's auc: 0.96879	valid_1's auc: 0.938889
[320]	valid_0's auc: 0.968861	valid_1's auc: 0.938764
[321]	valid_0's auc: 0.968987	valid_1's auc: 0.938831
[322]	valid_0's auc: 0.969098	valid_1's auc: 0.938981
[323]	valid_0's auc: 0.969202	valid_1's auc: 0.93903
[324]	valid_0's auc: 0.969261	valid_1's auc: 0.938997
[325]	valid_0's auc: 0.969353	valid_1's auc: 0.939022
[326]	valid_0's auc: 0.969542	valid_1's auc: 0.939047
[327]	valid_0's auc: 0.969695	valid_1's auc: 0.939039
[328]	valid_0's auc: 0.969748	valid_1's auc: 0.938972
[329]	valid_0's auc: 0.969869	valid_1's auc: 0.938997
[330]	valid_0's auc: 0.970037	valid_1's auc: 0.939014
[331]	valid_0's auc: 0.970112	valid_1's auc: 0.938881
[332]	valid_0's auc: 0.970257	valid_1's auc: 0.938989
[333]	valid_0's auc: 0.970395	

[475]	valid_0's auc: 0.982626	valid_1's auc: 0.939738
[476]	valid_0's auc: 0.982692	valid_1's auc: 0.939771
[477]	valid_0's auc: 0.982732	valid_1's auc: 0.939805
[478]	valid_0's auc: 0.982831	valid_1's auc: 0.939813
[479]	valid_0's auc: 0.982896	valid_1's auc: 0.939813
[480]	valid_0's auc: 0.982951	valid_1's auc: 0.939796
[481]	valid_0's auc: 0.98299	valid_1's auc: 0.939838
[482]	valid_0's auc: 0.983052	valid_1's auc: 0.939871
[483]	valid_0's auc: 0.983098	valid_1's auc: 0.939755
[484]	valid_0's auc: 0.983141	valid_1's auc: 0.939713
[485]	valid_0's auc: 0.983186	valid_1's auc: 0.939746
[486]	valid_0's auc: 0.983257	valid_1's auc: 0.939921
[487]	valid_0's auc: 0.9833	valid_1's auc: 0.939788
[488]	valid_0's auc: 0.983381	valid_1's auc: 0.93983
[489]	valid_0's auc: 0.983431	valid_1's auc: 0.939763
[490]	valid_0's auc: 0.983487	valid_1's auc: 0.93983
[491]	valid_0's auc: 0.98353	valid_1's auc: 0.939746
[492]	valid_0's auc: 0.983609	valid_1's auc: 0.93978
[493]	valid_0's auc: 0.983672	valid

...

/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['fzhydm', 'fzhyml_encoder', 'fzjglx', 'fzjgmc_encoder', 'fzjgsf', 'fzjgzt', 'hymldm_encoder', 'qyjglxmc_encoder', 'qylxmc_encoder', 'sshydm_encoder', 'xzgh', 'zczbbzmc_encoder', 'zl_encoder', 'zsszdsf_encoder']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	valid_0's auc: 0.501789	valid_1's auc: 0.499284
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's auc: 0.501789	valid_1's auc: 0.499284
[3]	valid_0's auc: 0.641244	valid_1's auc: 0.662393
[4]	valid_0's auc: 0.641244	valid_1's auc: 0.662393
[5]	valid_0's auc: 0.780554	valid_1's auc: 0.784435
[6]	valid_0's auc: 0.78011	valid_1's auc: 0.784599
[7]	valid_0's auc: 0.780056	valid_1's auc: 0.784742
[8]	valid_0's auc: 0.780056	valid_1's auc: 0.784742
[9]	valid_0's auc: 0.781682	valid_1's auc: 0.786091
[10]	valid_0's auc: 0.781752	valid_1's auc: 0.785947
[11]	valid_0's auc: 0.815036	valid_1's auc: 0.830566
[12]	valid_0's auc: 0.812873	valid_1's auc: 0.828561
[13]	valid_0's auc: 0.815816	valid_1's auc: 0.831384
[14]	valid_0's auc: 0.816054	valid_1's auc: 0.830996
[15]	valid_0's auc: 0.815325	valid_1's auc: 0.830356
[16]	valid_0's auc: 0.816106	valid_1's auc: 0.830979
[17]	valid_0's auc: 0.816948	valid_1's auc: 0.831611
[18]	valid_0's auc: 0.81813	valid_1's auc: 0.8

[155]	valid_0's auc: 0.936648	valid_1's auc: 0.902115
[156]	valid_0's auc: 0.937128	valid_1's auc: 0.902343
[157]	valid_0's auc: 0.937435	valid_1's auc: 0.902436
[158]	valid_0's auc: 0.937973	valid_1's auc: 0.902857
[159]	valid_0's auc: 0.938392	valid_1's auc: 0.902924
[160]	valid_0's auc: 0.938892	valid_1's auc: 0.903135
[161]	valid_0's auc: 0.939361	valid_1's auc: 0.903371
[162]	valid_0's auc: 0.939852	valid_1's auc: 0.903489
[163]	valid_0's auc: 0.940247	valid_1's auc: 0.903948
[164]	valid_0's auc: 0.940435	valid_1's auc: 0.903974
[165]	valid_0's auc: 0.94082	valid_1's auc: 0.90426
[166]	valid_0's auc: 0.94093	valid_1's auc: 0.904357
[167]	valid_0's auc: 0.94134	valid_1's auc: 0.904458
[168]	valid_0's auc: 0.941702	valid_1's auc: 0.904467
[169]	valid_0's auc: 0.941836	valid_1's auc: 0.904627
[170]	valid_0's auc: 0.942205	valid_1's auc: 0.904644
[171]	valid_0's auc: 0.942445	valid_1's auc: 0.904711
[172]	valid_0's auc: 0.942673	valid_1's auc: 0.904812
[173]	valid_0's auc: 0.943015	va

[309]	valid_0's auc: 0.968745	valid_1's auc: 0.910821
[310]	valid_0's auc: 0.96882	valid_1's auc: 0.910846
[311]	valid_0's auc: 0.968968	valid_1's auc: 0.910829
[312]	valid_0's auc: 0.969047	valid_1's auc: 0.910905
[313]	valid_0's auc: 0.969164	valid_1's auc: 0.910796
[314]	valid_0's auc: 0.969321	valid_1's auc: 0.911057
[315]	valid_0's auc: 0.969469	valid_1's auc: 0.911217
[316]	valid_0's auc: 0.969614	valid_1's auc: 0.91136
[317]	valid_0's auc: 0.969738	valid_1's auc: 0.911369
[318]	valid_0's auc: 0.969859	valid_1's auc: 0.911335
[319]	valid_0's auc: 0.970013	valid_1's auc: 0.91152
[320]	valid_0's auc: 0.970114	valid_1's auc: 0.91136
[321]	valid_0's auc: 0.970218	valid_1's auc: 0.911562
[322]	valid_0's auc: 0.97039	valid_1's auc: 0.911546
[323]	valid_0's auc: 0.970547	valid_1's auc: 0.911647
[324]	valid_0's auc: 0.970652	valid_1's auc: 0.911613
[325]	valid_0's auc: 0.970814	valid_1's auc: 0.911655
[326]	valid_0's auc: 0.970894	valid_1's auc: 0.911672
[327]	valid_0's auc: 0.971027	val

[468]	valid_0's auc: 0.982059	valid_1's auc: 0.911992
[469]	valid_0's auc: 0.982111	valid_1's auc: 0.912034
[470]	valid_0's auc: 0.982131	valid_1's auc: 0.912018
[471]	valid_0's auc: 0.982214	valid_1's auc: 0.912009
[472]	valid_0's auc: 0.982265	valid_1's auc: 0.912034
[473]	valid_0's auc: 0.982308	valid_1's auc: 0.911975
[474]	valid_0's auc: 0.982376	valid_1's auc: 0.911959
[475]	valid_0's auc: 0.982428	valid_1's auc: 0.911984
[476]	valid_0's auc: 0.982486	valid_1's auc: 0.911916
[477]	valid_0's auc: 0.982568	valid_1's auc: 0.91195
[478]	valid_0's auc: 0.982626	valid_1's auc: 0.91195
[479]	valid_0's auc: 0.982689	valid_1's auc: 0.91195
[480]	valid_0's auc: 0.982775	valid_1's auc: 0.911891
[481]	valid_0's auc: 0.982855	valid_1's auc: 0.911891
[482]	valid_0's auc: 0.982935	valid_1's auc: 0.911824
[483]	valid_0's auc: 0.982984	valid_1's auc: 0.911841
[484]	valid_0's auc: 0.983042	valid_1's auc: 0.911942
[485]	valid_0's auc: 0.983102	valid_1's auc: 0.911857
[486]	valid_0's auc: 0.983152	v

In [118]:
test_df['FORTARGET1'] = test_df['FORTARGET1_round0'] * 0.25 + test_df['FORTARGET1_round1'] * 0.25 + test_df['FORTARGET1_round2'] * 0.25 + test_df['FORTARGET1_round3'] * 0.25


In [119]:
print(np.mean(test_df['FORTARGET1']))

def shixin_after_deal(df):
    FORTARGET1 = df['FORTARGET1']
    is_suzhou = df['is_suzhou']
    is_fr = df['is_fr']
    is_fzc = df['is_fzc']
    is_fz = df['is_fz']
    is_qq = df['is_qq']
    is_zp = df['is_zp']
    is_tyshxydm_many = df['is_tyshxydm_many']
    is_zch_many = df['is_zch_many']
    if (is_suzhou == 0) | (is_fr == 1):
        FORTARGET1 = 0
    if is_fzc == 1:
        FORTARGET1 = FORTARGET1 / 1.5
    if is_fz:
        FORTARGET1 = FORTARGET1 / 3
    if is_qq:
        FORTARGET1 = FORTARGET1 / 2
    if is_zp:
        FORTARGET1 = FORTARGET1 / 1.5
    if is_tyshxydm_many > 1:
        FORTARGET1 = 0
    if is_zch_many > 1:
        FORTARGET1 = 0
    return FORTARGET1

test_df['FORTARGET1'] = test_df.apply(shixin_after_deal, axis=1)
print(np.mean(test_df['FORTARGET1']))


0.03996555746751438
0.036476433960792795


In [120]:
train_df['chufa_weight'] = 1
train_df.loc[train_df.is_chufa == 1, 'chufa_weight'] = 2


In [121]:
chufa_fea = ['zczj', 'fzjgsf', 'fzjgzt', 'fzhydm', 'fzjgqx', 'fzjglx', 'sjly', 'rwbh', 
#               'zckbzj', 
       'QYFZCHRD_yynx', 'sshydm_encoder', 'xzgh', 'qyjd', 'qywd', 
       'QYBZRYXX_number', 'QYFZCHRD_number', 'QYSWDJXX_number', 'FRXZXKZX_number',
       'SDBDJQQJMCF_number', 'SGSFRXZXKXX_number', 'XKZZNJXX_number',
       'ZPSJ_number', 'ZZDJBGXX_number', 'QYJL_number', 
             
      'zczbbzmc_encoder', 'hymldm_encoder', 'zsszdsf_encoder', 'fzhyml_encoder',
       'xxtgbmmc_encoder', 'zl_encoder', 'sszj_encoder', 'fzjgmc_encoder',
       'qyjglxmc_encoder', 'qylxmc_encoder',
             
             'QYFZCHRD_swglm_encoder',
       'QYFZCHRD_gljg_encoder', 'QYFZCHRD_djzclx_encoder', 
        'SGSFRXZXKXX_dfbm', 'SGSFRXZXKXX_xknr_encoder', 'SGSFRXZXKXX_xmmc_encoder', 'SGSFRXZXKXX_splb_encoder', 
       'SGSFRXZXKXX_xkjg_encoder', 'SGSFRXZXKXX_xysyfw_encoder', 'SGSFRXZXKXX_sjzt2_encoder', 
        'XKZZNJXX_njnd_encoder', 'XKZZNJXX_njjgmc_encoder', 'XKZZNJXX_njsxmc_encoder', 
         'ZZDJBGXX_zzmc_encoder', 'ZZDJBGXX_zyfw_encoder', 'ZZDJBGXX_rdjgqc_encoder', 'ZZDJBGXX_zzzsbh_encoder', 
      'QY_tzrgs', 'QY_zwzls', 'QY_fddbrbz_gs', 'QY_sxdbbz_gs', 
              
        'fzsw_deltaDay', 'clhz_deltaDay', 
       'clrq_month', 'clrq_day', 'hzrq_year', 'hzrq_month', 'hzrq_day', 'clrq_year',
              
             'jgdzzs_top3_encoder', 'jgdzzs_top6_encoder',
 'QYSWDJXX_fddbrzjmc_encoder', 'QYSWDJXX_shdw_encoder',
 'QYSWDJXX_qy_encoder', 'QYSWDJXX_shjg_encoder', 'QYSWDJXX_djzclx_encoder',
             
#        'ZPSJ_zprs_max', 'ZPSJ_zprs_min', 'ZPSJ_zprs_mean', 'ZPSJ_zprs_std',
              
       'jgdzzs_encoder', 'jyfw_encoder', 'sshymc_encoder',
             
             'jyfw_array_0_fea', 'jyfw_array_1_fea', 'jyfw_array_2_fea',
             'jyfw_array_3_fea', 'jyfw_array_4_fea', 'jyfw_array_5_fea',
             'jyfw_array_6_fea', 'jyfw_array_7_fea', 'jyfw_array_8_fea',
             'jyfw_array_9_fea',

#              'QYFZCHRD_zcdz_encoder', 
#     'XKZZNJXX_xxtgbmmc_encoder', 
#      'QYFZCHRD_rwbh_encoder', 'QYFZCHRD_xxtgbmmc_encoder',
#     'SGSFRXZXKXX_xzxkbm_encoder', 'SGSFRXZXKXX_xkjdswh_encoder', 'SGSFRXZXKXX_xxtgbmmc_encoder',
             
#        'ZZDJBGXX_jzsx_deltaDay', 'zzsxcl_deltaDay', 'zzjzcl_deltaDay', 'zzsxhz_deltaDay', 'zzjzhz_deltaDay', 
             
#       'ZPSJ_zylb_kinds', 'ZPSJ_gzjy_kinds',
#        'ZPSJ_gzdd_kinds', 'ZPSJ_zprs_kinds', 'ZPSJ_zwyx_kinds',
#        'ZPSJ_zdxl_kinds', 'ZPSJ_zprq_kinds',
              
#         'SGSFRXZXKXX_xkjdjz_deltaDay',
#       'SGSFRXZXKXX_xkjzq_year', 'SGSFRXZXKXX_xkjzq_month', 'SGSFRXZXKXX_xkjzq_day', 
#       'SGSFRXZXKXX_xkjdrq_year', 'SGSFRXZXKXX_xkjdrq_month', 'SGSFRXZXKXX_xkjdrq_day',  
    
       
#        'clxkjz_deltaDay', 'hzxkjz_deltaDay', 'clxkjd_deltaDay', 'hzxkjd_deltaDay', 
#         'njhz_deltaDay', 'njcl_deltaDay', 
#        'XKZZNJXX_njrq_year', 'XKZZNJXX_njrq_month', 'XKZZNJXX_njrq_day', 
#        'ZZDJBGXX_zzsxq_year', 'ZZDJBGXX_zzsxq_month', 'ZZDJBGXX_zzsxq_day', 
#        'ZZDJBGXX_zzjzq_year', 'ZZDJBGXX_zzjzq_month', 'ZZDJBGXX_zzjzq_day', 
       
#        'czbl_max', 'czbl_min', 'czbl_mean', 'czbl_std',
             
       'fzrq_encoder', 'bghzrq_encoder',
     'gszch_encoder', 'fddbrzjhm_encoder', 'zzjgdm_encoder',
       
#        'ZPSJ_wzmc_max', 'ZPSJ_wzmc_min', 'ZPSJ_wzmc_mean',
#  'ZPSJ_wzmc_std', 'ZPSJ_gzjy_max', 'ZPSJ_gzjy_min', 'ZPSJ_gzjy_mean',
#  'ZPSJ_gzjy_std', 'ZPSJ_gzdd_max', 'ZPSJ_gzdd_min', 'ZPSJ_gzdd_mean',
#  'ZPSJ_gzdd_std', 'ZPSJ_zylb_max', 'ZPSJ_zylb_min', 'ZPSJ_zylb_mean',
#  'ZPSJ_zylb_std', 'ZPSJ_zwyx_max', 'ZPSJ_zwyx_min', 'ZPSJ_zwyx_mean',
#  'ZPSJ_zwyx_std', 'ZPSJ_zdxl_max', 'ZPSJ_zdxl_min', 'ZPSJ_zdxl_mean',
#  'ZPSJ_zdxl_std', 'ZPSJ_ywzj_max', 'ZPSJ_ywzj_min', 'ZPSJ_ywzj_mean',
#  'ZPSJ_ywzj_std',
       
#        'SGSFRXZXKXX_is_shixin_sjly_rate', 'SGSFRXZXKXX_is_chufa_sjly_rate',
#  'SGSFRXZXKXX_is_shixin_cjrID_rate', 'SGSFRXZXKXX_is_chufa_cjrID_rate',
#  'SGSFRXZXKXX_is_shixin_xxtgbmbm_rate',
#  'SGSFRXZXKXX_is_chufa_xxtgbmbm_rate', 'SGSFRXZXKXX_is_shixin_splb_rate',
#  'SGSFRXZXKXX_is_chufa_splb_rate', 'SGSFRXZXKXX_is_shixin_dfbm_rate',
#  'SGSFRXZXKXX_is_chufa_dfbm_rate', 'SGSFRXZXKXX_is_shixin_xkjg_rate',
#  'SGSFRXZXKXX_is_chufa_xkjg_rate',
       
      ]



In [122]:
lgb_model_chufa = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=127, reg_alpha=6, reg_lambda=10, max_bin=425,
    max_depth=-1, n_estimators=5000, objective='binary',
#     subsample=0.8, 
    colsample_bytree=1, subsample_freq=1,
    learning_rate=0.05, random_state=2018, n_jobs=-1,min_child_samples=5
)

test_df['FORTARGET2'] = 0

skf = StratifiedKFold(n_splits=15, random_state=2018, shuffle=True)
best_score = []
for index, (train_index, test_index) in enumerate(skf.split(train_df[chufa_fea], train_df['is_chufa'])):
    weight = train_df['chufa_weight'].iloc[train_index].tolist()
    lgb_model_chufa.fit(train_df[chufa_fea].iloc[train_index], train_df['is_chufa'][train_index],
                        sample_weight=weight,
                        categorical_feature=categorical_feature_chufa,
                        eval_set=[(train_df[chufa_fea].iloc[train_index], train_df['is_chufa'][train_index]),
                            (train_df[chufa_fea].iloc[test_index], train_df['is_chufa'][test_index])], early_stopping_rounds=100, eval_metric='auc')
    best_score.append(lgb_model_chufa.best_score_['valid_1']['auc'])
    print(best_score)
    test_pred = lgb_model_chufa.predict_proba(test_df[chufa_fea], num_iteration=lgb_model_shixin.best_iteration_)[:, 1]
    print('test mean:', test_pred.mean())
    test_df['FORTARGET2'] = test_df['FORTARGET2'] + test_pred
print(np.mean(best_score))

test_df['FORTARGET2'] = test_df['FORTARGET2'] / 15
mean = test_df['FORTARGET2'].mean()
print('mean:', mean)


/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['fzhydm', 'fzhyml_encoder', 'fzjglx', 'fzjgmc_encoder', 'fzjgsf', 'fzjgzt', 'hymldm_encoder', 'qyjglxmc_encoder', 'qylxmc_encoder', 'sshydm_encoder', 'xzgh', 'zczbbzmc_encoder', 'zl_encoder', 'zsszdsf_encoder']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	valid_0's auc: 0.937007	valid_1's auc: 0.940095
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.938998	valid_1's auc: 0.94304
[3]	valid_0's auc: 0.940505	valid_1's auc: 0.945258
[4]	valid_0's auc: 0.941597	valid_1's auc: 0.946559
[5]	valid_0's auc: 0.942512	valid_1's auc: 0.947104
[6]	valid_0's auc: 0.943679	valid_1's auc: 0.94811
[7]	valid_0's auc: 0.944442	valid_1's auc: 0.949561
[8]	valid_0's auc: 0.944844	valid_1's auc: 0.950108
[9]	valid_0's auc: 0.945219	valid_1's auc: 0.950623
[10]	valid_0's auc: 0.945639	valid_1's auc: 0.951337
[11]	valid_0's auc: 0.946886	valid_1's auc: 0.952048
[12]	valid_0's auc: 0.947218	valid_1's auc: 0.952499
[13]	valid_0's auc: 0.947832	valid_1's auc: 0.953267
[14]	valid_0's auc: 0.948329	valid_1's auc: 0.953539
[15]	valid_0's auc: 0.950016	valid_1's auc: 0.954757
[16]	valid_0's auc: 0.951277	valid_1's auc: 0.955144
[17]	valid_0's auc: 0.952524	valid_1's auc: 0.956794
[18]	valid_0's auc: 0.953229	valid_1's auc: 0.9

[156]	valid_0's auc: 0.986324	valid_1's auc: 0.967201
[157]	valid_0's auc: 0.986429	valid_1's auc: 0.967197
[158]	valid_0's auc: 0.986543	valid_1's auc: 0.967216
[159]	valid_0's auc: 0.986656	valid_1's auc: 0.967215
[160]	valid_0's auc: 0.98676	valid_1's auc: 0.967236
[161]	valid_0's auc: 0.986846	valid_1's auc: 0.967314
[162]	valid_0's auc: 0.986952	valid_1's auc: 0.967366
[163]	valid_0's auc: 0.98705	valid_1's auc: 0.967323
[164]	valid_0's auc: 0.987163	valid_1's auc: 0.96732
[165]	valid_0's auc: 0.987267	valid_1's auc: 0.967244
[166]	valid_0's auc: 0.987374	valid_1's auc: 0.967202
[167]	valid_0's auc: 0.987477	valid_1's auc: 0.967235
[168]	valid_0's auc: 0.987579	valid_1's auc: 0.967243
[169]	valid_0's auc: 0.987684	valid_1's auc: 0.967262
[170]	valid_0's auc: 0.987785	valid_1's auc: 0.967251
[171]	valid_0's auc: 0.987868	valid_1's auc: 0.96726
[172]	valid_0's auc: 0.987965	valid_1's auc: 0.967231
[173]	valid_0's auc: 0.988058	valid_1's auc: 0.967241
[174]	valid_0's auc: 0.988166	va

[311]	valid_0's auc: 0.995489	valid_1's auc: 0.967415
[312]	valid_0's auc: 0.995517	valid_1's auc: 0.9674
[313]	valid_0's auc: 0.995547	valid_1's auc: 0.967459
[314]	valid_0's auc: 0.995574	valid_1's auc: 0.967436
[315]	valid_0's auc: 0.995603	valid_1's auc: 0.967374
[316]	valid_0's auc: 0.99563	valid_1's auc: 0.967363
[317]	valid_0's auc: 0.995654	valid_1's auc: 0.967434
[318]	valid_0's auc: 0.995682	valid_1's auc: 0.967435
[319]	valid_0's auc: 0.995719	valid_1's auc: 0.967394
[320]	valid_0's auc: 0.995746	valid_1's auc: 0.967426
[321]	valid_0's auc: 0.995778	valid_1's auc: 0.96738
[322]	valid_0's auc: 0.995805	valid_1's auc: 0.967411
[323]	valid_0's auc: 0.995831	valid_1's auc: 0.967394
[324]	valid_0's auc: 0.995856	valid_1's auc: 0.967382
[325]	valid_0's auc: 0.995881	valid_1's auc: 0.967348
[326]	valid_0's auc: 0.995901	valid_1's auc: 0.967372
[327]	valid_0's auc: 0.995935	valid_1's auc: 0.967361
[328]	valid_0's auc: 0.995969	valid_1's auc: 0.96734
[329]	valid_0's auc: 0.995997	val

/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['fzhydm', 'fzhyml_encoder', 'fzjglx', 'fzjgmc_encoder', 'fzjgsf', 'fzjgzt', 'hymldm_encoder', 'qyjglxmc_encoder', 'qylxmc_encoder', 'sshydm_encoder', 'xzgh', 'zczbbzmc_encoder', 'zl_encoder', 'zsszdsf_encoder']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


...

[155]	valid_0's auc: 0.986483	valid_1's auc: 0.962937
[156]	valid_0's auc: 0.986599	valid_1's auc: 0.962933
[157]	valid_0's auc: 0.9867	valid_1's auc: 0.962944
[158]	valid_0's auc: 0.986823	valid_1's auc: 0.962941
[159]	valid_0's auc: 0.986933	valid_1's auc: 0.962946
[160]	valid_0's auc: 0.987054	valid_1's auc: 0.962898
[161]	valid_0's auc: 0.987155	valid_1's auc: 0.962874
[162]	valid_0's auc: 0.987259	valid_1's auc: 0.962866
[163]	valid_0's auc: 0.987372	valid_1's auc: 0.962882
[164]	valid_0's auc: 0.98749	valid_1's auc: 0.962848
[165]	valid_0's auc: 0.987587	valid_1's auc: 0.962823
[166]	valid_0's auc: 0.987704	valid_1's auc: 0.962742
[167]	valid_0's auc: 0.987783	valid_1's auc: 0.962737
[168]	valid_0's auc: 0.98788	valid_1's auc: 0.962772
[169]	valid_0's auc: 0.987982	valid_1's auc: 0.962792
[170]	valid_0's auc: 0.98807	valid_1's auc: 0.962862
[171]	valid_0's auc: 0.988175	valid_1's auc: 0.962925
[172]	valid_0's auc: 0.988275	valid_1's auc: 0.96295
[173]	valid_0's auc: 0.988371	vali

/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overridden. New categorical_feature is ['fzhydm', 'fzhyml_encoder', 'fzjglx', 'fzjgmc_encoder', 'fzjgsf', 'fzjgzt', 'hymldm_encoder', 'qyjglxmc_encoder', 'qylxmc_encoder', 'sshydm_encoder', 'xzgh', 'zczbbzmc_encoder', 'zl_encoder', 'zsszdsf_encoder']
  warnings.warn('categorical_feature in Dataset is overridden. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	valid_0's auc: 0.936285	valid_1's auc: 0.92429
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.940309	valid_1's auc: 0.930297
[3]	valid_0's auc: 0.941498	valid_1's auc: 0.930562
[4]	valid_0's auc: 0.942472	valid_1's auc: 0.93138
[5]	valid_0's auc: 0.943405	valid_1's auc: 0.932411
[6]	valid_0's auc: 0.944485	valid_1's auc: 0.932487
[7]	valid_0's auc: 0.94489	valid_1's auc: 0.933254
[8]	valid_0's auc: 0.945796	valid_1's auc: 0.934123
[9]	valid_0's auc: 0.946195	valid_1's auc: 0.934979
[10]	valid_0's auc: 0.947317	valid_1's auc: 0.935459
[11]	valid_0's auc: 0.947935	valid_1's auc: 0.936621
[12]	valid_0's auc: 0.948876	valid_1's auc: 0.937824
[13]	valid_0's auc: 0.949438	valid_1's auc: 0.938202
[14]	valid_0's auc: 0.949805	valid_1's auc: 0.938646
[15]	valid_0's auc: 0.94992	valid_1's auc: 0.938812
[16]	valid_0's auc: 0.951664	valid_1's auc: 0.940033
[17]	valid_0's auc: 0.952701	valid_1's auc: 0.940679
[18]	valid_0's auc: 0.953948	valid_1's auc: 0.941

[156]	valid_0's auc: 0.986729	valid_1's auc: 0.954787
[157]	valid_0's auc: 0.98683	valid_1's auc: 0.954908
[158]	valid_0's auc: 0.986942	valid_1's auc: 0.954862
[159]	valid_0's auc: 0.98705	valid_1's auc: 0.954873
[160]	valid_0's auc: 0.987144	valid_1's auc: 0.954841
[161]	valid_0's auc: 0.987253	valid_1's auc: 0.954842
[162]	valid_0's auc: 0.987352	valid_1's auc: 0.954813
[163]	valid_0's auc: 0.987448	valid_1's auc: 0.954807
[164]	valid_0's auc: 0.987547	valid_1's auc: 0.954772
[165]	valid_0's auc: 0.987651	valid_1's auc: 0.954765
[166]	valid_0's auc: 0.987743	valid_1's auc: 0.954689
[167]	valid_0's auc: 0.987826	valid_1's auc: 0.954648
[168]	valid_0's auc: 0.987935	valid_1's auc: 0.954693
[169]	valid_0's auc: 0.988025	valid_1's auc: 0.954696
[170]	valid_0's auc: 0.988128	valid_1's auc: 0.954681
[171]	valid_0's auc: 0.988234	valid_1's auc: 0.95465
[172]	valid_0's auc: 0.98832	valid_1's auc: 0.954641
[173]	valid_0's auc: 0.988414	valid_1's auc: 0.954608
[174]	valid_0's auc: 0.98851	val

[309]	valid_0's auc: 0.995567	valid_1's auc: 0.954757
[310]	valid_0's auc: 0.995591	valid_1's auc: 0.954741
[311]	valid_0's auc: 0.995617	valid_1's auc: 0.954673
[312]	valid_0's auc: 0.995639	valid_1's auc: 0.954706
[313]	valid_0's auc: 0.995668	valid_1's auc: 0.954677
[314]	valid_0's auc: 0.995696	valid_1's auc: 0.954635
[315]	valid_0's auc: 0.995721	valid_1's auc: 0.95463
[316]	valid_0's auc: 0.995748	valid_1's auc: 0.954647
[317]	valid_0's auc: 0.995776	valid_1's auc: 0.954661
[318]	valid_0's auc: 0.995795	valid_1's auc: 0.95463
[319]	valid_0's auc: 0.995822	valid_1's auc: 0.95465
[320]	valid_0's auc: 0.995853	valid_1's auc: 0.954644
[321]	valid_0's auc: 0.995877	valid_1's auc: 0.954678
[322]	valid_0's auc: 0.995907	valid_1's auc: 0.95469
[323]	valid_0's auc: 0.995934	valid_1's auc: 0.954698
[324]	valid_0's auc: 0.995957	valid_1's auc: 0.954688
[325]	valid_0's auc: 0.99598	valid_1's auc: 0.95473
[326]	valid_0's auc: 0.996	valid_1's auc: 0.95479
[327]	valid_0's auc: 0.99602	valid_1's

In [123]:
xgb_online = pd.read_csv('../temp/xgb_online.csv')

test_df.rename(columns={'FORTARGET1':'lgb_FORTARGET1', 'FORTARGET2':'lgb_FORTARGET2'}, inplace=True)

test_df['xgb_FORTARGET1'] = xgb_online['FORTARGET1']
test_df['xgb_FORTARGET2'] = xgb_online['FORTARGET2']

print(test_df[['lgb_FORTARGET1', 'lgb_FORTARGET2', 'xgb_FORTARGET1', 'xgb_FORTARGET2']].head())


   lgb_FORTARGET1  lgb_FORTARGET2  xgb_FORTARGET1  xgb_FORTARGET2
0        0.032177        0.068973        0.012917        0.032887
1        0.046322        0.341820        0.008110        0.355273
2        0.066296        0.277929        0.024177        0.248291
3        0.042292        0.022163        0.021981        0.014348
4        0.003318        0.025941        0.000223        0.014174


In [124]:
test_df['FORTARGET1'] = test_df['xgb_FORTARGET1'] * 0.6 + test_df['lgb_FORTARGET1'] * 0.4

test_df['FORTARGET2'] = test_df['xgb_FORTARGET2'] * 0.2 + test_df['lgb_FORTARGET2'] * 0.8



In [125]:
test_df.rename(columns={'qymc':'EID'}, inplace=True)
print(test_df[['EID', 'FORTARGET1', 'FORTARGET2']].head())


                                EID  FORTARGET1  FORTARGET2
0  C06A265AA0BAF3EEF90CB893F22A355C    0.020621    0.061756
1  057B332E88A16EA4BD4C662F9281E708    0.023395    0.344511
2  4B70FC35C51C07048E98C44698BF2123    0.041024    0.272001
3  FFDA9306B3E914BF67ACE0A99BF47DA6    0.030106    0.020600
4  CF94736A17FE3078D62C5C2BBEF01033    0.001461    0.023588


In [126]:
# 导出预测结果
def exportResult(df, fileName):
    df.to_csv('../result/%s.csv' % fileName, header=True, index=False)
    
exportResult(test_df[['EID', 'FORTARGET1', 'FORTARGET2']], 'compliance_assessment')
